# 1. Import and Install Dependencies


In [ ]:
!pip install opencv-python mediapipe sklearn matplotlib


In [ ]:
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using Media Pipe Holistic

In [ ]:

mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [28]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [30]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections
    Extract_points(results)
    

In [6]:
def draw_styled_landmarks(image, results):
    
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# Function for Extracting Key Points

In [ ]:
def Extract_KeyPoints(results):

    
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,lh, rh])A

# 3) Data Collection 

In [3]:

# Paths and parameters
DATA_PATH = r'E:\Sign lagnuage'  # Path to the folder containing videos
OUTPUT_PATH = r'E:\InputData'  # Path to the folder for storing keypoints data
for video_file in os.listdir(DATA_PATH):
    video_path = os.path.join(DATA_PATH, video_file)
    if not video_file.endswith('.mp4'):  # Check if it's a video file
        continue

    # Create a folder for each video file in the output path
    output_video_folder = os.path.splitext(video_file)[0]  # Extract video file name without extension
    output_video_path = os.path.join(OUTPUT_PATH, output_video_folder)
    os.makedirs(output_video_path, exist_ok=True)

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        continue
    
    # Get total number of frames
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Initialize mediapipe holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

        # Loop through video frames
        for frame_num in range(total_frames):
            ret, frame = cap.read()
            if not ret:
                print(f"Error: Could not read frame {frame_num} from {video_path}")
                break

            # Make detections
            image, results = mediapipe_detection(frame, holistic)

            # Export keypoints
            keypoints = Extract_KeyPoints(results)
            npy_path = os.path.join(output_video_path, f'{frame_num}.npy')
            np.save(npy_path, keypoints)

        # Release video capture
        cap.release()
